In [1]:
import pandas as pd
from pgmpy.models import BayesianNetwork
from pgmpy.estimators import MaximumLikelihoodEstimator
from pgmpy.inference import VariableElimination

/home/sven/dev/DHBW-KI-Laborarbeit/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from data_cleaning.data_cleaning import get_zimmer, get_stockwerk, get_heizung, get_kindergarten, get_schule, get_bahn, get_miete, get_nebenkosten, get_alter, get_lage, get_kaution, get_kueche, get_bad, get_mobliert, get_quadratmeter

file_path = '../data/Wohnungen_1.csv'  # Gegebenen Datensatz einlesen
data = pd.read_csv(file_path, sep=';')

# Bewohnerkategorie bestimmen
resident_columns = ['Studierende', 'Kleinfamilie', 'DINK', 'Alleinerziehende', 'Expatriate', 'Rentnerpaar']
data['Bewohnerkategorie'] = data[resident_columns].apply(lambda row: 'Keine' if all(row == 'nein') else row.idxmax(), axis=1)

# Bereinigung der Daten
data = data[data['Bewohnerkategorie'] != 'Keine']

# Spalten, in denen "nein" durch 0 und "ja" durch 1 ersetzt werden sollen
columns_to_replace = ['Hausmeister', 'Garage', 'Aufzug', 'Balkon', 'Terrasse', "Kehrwoche", "Studierende", "Kleinfamilie", "DINK", "Alleinerziehende", "Expatriate", "Rentnerpaar"]

# Ersetze "nein" und "ja" in den angegebenen Spalten
data[columns_to_replace] = data[columns_to_replace].replace({'nein': 0, 'ja': 1})

data['Zimmerzahl'] = data['Zimmerzahl'].apply(get_zimmer)
data['Stockwerk'] = data['Stockwerk'].apply(get_stockwerk)
data['Heizung'] = data['Heizung'].apply(get_heizung)
data['Kindergarten'] = data['Kindergarten'].apply(get_kindergarten)
data['Schule'] = data['Schule'].apply(get_schule)
data['S-Bahn'] = data['S-Bahn'].apply(get_bahn)
data['Miete'] = data['Miete'].apply(get_miete)
data['Nebenkosten'] = data['Nebenkosten'].apply(get_nebenkosten)
data['Alter'] = data['Alter'].apply(get_alter)
data['Lage'] = data['Lage'].apply(get_lage)
data['Kaution'] = data['Kaution'].apply(get_kaution)
data['Kueche'] = data['Kueche'].apply(get_kueche)
data['Bad'] = data['Bad'].apply(get_bad)
data['Moebliert'] = data['Moebliert'].apply(get_mobliert)
data['Quadratmeter'] = data['Quadratmeter'].apply(get_quadratmeter)

print(data.head())

    Zimmerzahl  Stockwerk  Heizung  Hausmeister  Kindergarten  Schule  S-Bahn  \
11           2          0        0            0             1       2       4   
13           2          0        5            0             2       2       3   
23           2          1        4            1             1       2       4   
27           0          0        1            0             1       0       0   
40           0          8        5            0             0       2       2   

    Garage  Miete  Nebenkosten  ...  Kehrwoche  Moebliert  Quadratmeter  \
11       0    726          276  ...          0          0           116   
13       1    726          226  ...          0          0           106   
23       0    776          276  ...          0          0           116   
27       0    226           50  ...          0          0            25   
40       0    226           50  ...          0          0            36   

    Studierende  Kleinfamilie  DINK  Alleinerziehende  Expatri

/tmp/ipykernel_2342/1856597896.py:17: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data[columns_to_replace] = data[columns_to_replace].replace({'nein': 0, 'ja': 1})


In [5]:
# data['Wohnlage'] = data[['Lage', 'S-Bahn']].mean(axis=1)
# data['Wohnkomfort'] = data[['Balkon', 'Terrasse', 'Aufzug', 'Kueche', 'Bad', 'Garage', 'Moebliert', 'Alter']].mean(axis=1)
# data['Wohnkosten'] = data[['Miete', 'Nebenkosten', 'Kaution']].mean(axis=1)
# data['Bildung'] = data[['Kindergarten', 'Schule']].mean(axis=1)
# data['Raum'] = data[['Zimmerzahl', 'Stockwerk', 'Heizung']].mean(axis=1)


model = BayesianNetwork([
    ('Zimmerzahl', 'Bewohnerkategorie'),
    ('Stockwerk', 'Bewohnerkategorie'),
    ('Heizung', 'Bewohnerkategorie'),
    ('Hausmeister', 'Bewohnerkategorie'),
    ('Kindergarten', 'Bewohnerkategorie'),
    ('Schule', 'Bewohnerkategorie'),
    ('S-Bahn', 'Bewohnerkategorie'),
    ('Garage', 'Bewohnerkategorie'),
    ('Miete', 'Bewohnerkategorie'),
    ('Nebenkosten', 'Bewohnerkategorie'),
    ('Alter', 'Bewohnerkategorie'),
    ('Aufzug', 'Bewohnerkategorie'),
    ('Lage', 'Bewohnerkategorie'),
    ('Kaution', 'Bewohnerkategorie'),
    ('Kueche', 'Bewohnerkategorie'),
    ('Bad', 'Bewohnerkategorie'),
    ('Balkon', 'Bewohnerkategorie'),
    ('Terrasse', 'Bewohnerkategorie'),
    ('Kehrwoche', 'Bewohnerkategorie'),
    ('Moebliert', 'Bewohnerkategorie'),
    ('Quadratmeter', 'Bewohnerkategorie'),
    # ('Wohnlage', 'Bewohnerkategorie'),
    # ('Wohnkomfort', 'Bewohnerkategorie'),
    # ('Wohnkosten', 'Bewohnerkategorie'),
    # ('Bildung', 'Bewohnerkategorie'),
    # ('Raum', 'Bewohnerkategorie'),
])

# Schritt 4: Training des Bayes-Netzes
model.fit(data, estimator=MaximumLikelihoodEstimator)


In [13]:
# Schritt 5: Inferenz durchführen
inference = VariableElimination(model)

# Beispiel-Eingabe: Teilweise bekannte Daten
example_input = {
    "Quadratmeter": 100,
    "Lage": 1,
}

# Wahrscheinlichkeiten für jede Zielklasse berechnen
prediction = inference.map_query(variables=["Bewohnerkategorie"], evidence=example_input)


print("Klassifikationsergebnis:")
print(prediction)

Eliminating: Terrasse: 100%|██████████| 3/3 [00:00<00:00, 3560.53it/s]

Klassifikationsergebnis:
{'Bewohnerkategorie': 'Studierende'}
